# Notebook Imports

In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import date
import seaborn as sns
from scipy import stats
import umap

import matplotlib.pyplot as plt
import warnings

from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.feature_selection import VarianceThreshold
from sklearn.manifold import TSNE

warnings.filterwarnings('ignore')

# Constants

In [ ]:
FILE = '/kaggle/input/data-science-for-good/Dept_49-00035/49-00035_Incidents_2016.csv'

# Load Data

In [ ]:
data = pd.read_csv(FILE, low_memory=False)
df = pd.DataFrame(data)

df.drop([0], axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)

df.head()

# Step 1: Data Exploration and Preprocessing

In [ ]:
print(f"\nThe shape of the data: {df.shape}\n")
print(f"The types of the data: \n{df.dtypes}\n")
print(f"The empty row in the data: \n{df.isnull().sum()}")

In [ ]:
df['INCIDENT_DATE'].replace(to_replace=np.nan, value=0, inplace=True)

In [ ]:
# Extracting dates

df['INCIDENT_DATE'] = pd.to_datetime(df.INCIDENT_DATE, format="mixed")
df["INCIDENT_YEAR"] = df["INCIDENT_DATE"].dt.strftime('%Y')
df["INCIDENT_MONTH"] = df["INCIDENT_DATE"].dt.strftime('%m')
df["INCIDENT_WEEKDAY"] = df["INCIDENT_DATE"].dt.strftime('%w')
df["INCIDENT_TIME"] = df["INCIDENT_DATE"].dt.strftime("%H:%M")
df['INCIDENT_DATE'] = df['INCIDENT_DATE'].astype(str)
df["INCIDENT_DATE"] = df["INCIDENT_DATE"].str.split(" ").str[0].str.split("-").str[2]  # Corrected date splitting

In [ ]:
df.head()

### Handling missing values in the dataframe

In [ ]:
for feature in ['INCIDENT_DATE', "INCIDENT_YEAR", "INCIDENT_MONTH", 'INCIDENT_WEEKDAY']:
    df[feature].replace(to_replace=np.nan, value=0, inplace=True)
    df[feature] = df[feature].astype(int)
    value = round(df[feature].mean())
    df[feature].replace(to_replace=np.nan, value=value, inplace=True)

In [ ]:
df['INCIDENT_YEAR'] = df['INCIDENT_YEAR'].astype(int)
replace = df[(df['INCIDENT_YEAR'] < 2015) | (df['INCIDENT_YEAR'] > 2018)]['INCIDENT_YEAR'].tolist()
value = round(df[(df['INCIDENT_YEAR'] >= 2015) | (df['INCIDENT_YEAR'] <= 2018)]['INCIDENT_YEAR'].mean())
df['INCIDENT_YEAR'].replace(to_replace=replace, value=value, inplace=True)

In [ ]:
for feature in df.columns:
    df[feature].replace(to_replace=np.nan, value="UNKNOWN", inplace=True) 

In [ ]:
responses = df['INCIDENT_TIME'].unique().tolist()

times = {
    '00:00': '00:',
    '01:00': '01:',
    '02:00': '02:',
    '03:00': '03:',
    '04:00': '04:',
    '05:00': '05:',
    '06:00': '06:',
    '07:00': '07:',
    '08:00': '08:',
    '09:00': '09:',
    '10:00': '10:',
    '11:00': '11:',
    '12:00': '12:',
    '13:00': '13:',
    '14:00': '14:',
    '15:00': '15:',
    '16:00': '16:',
    '17:00': '17:',
    '18:00': '18:',
    '19:00': '19:',
    '20:00': '20:',
    '21:00': '21:',
    '22:00': '22:',
    '23:00': '23:',
}

In [ ]:
for response in responses:
        for key, values in times.items():            
            if values in response:
                df['INCIDENT_TIME'].replace(response, value=key, inplace=True)                 
                break

In [ ]:
print(f"The empty row in the data: \n{df.isnull().sum()}")

In [ ]:
df.shape

In [ ]:
df['INCIDENT_YEAR'].value_counts()

In [ ]:
df.drop(['INCIDENT_UNIQUE_IDENTIFIER', 'LOCATION_FULL_STREET_ADDRESS_OR_INTERSECTION'], axis=1, inplace=True)

# Step 2: Data Visualization

In [ ]:
# Pie Chart

# Creating data
data = [len(df[df['INCIDENT_YEAR'] == feature]) for feature in df['INCIDENT_YEAR'].value_counts().index]
labels = df['INCIDENT_YEAR'].value_counts().index[0:5]

colors = ['#A0D568', '#FFCE54'] # creating color parameters
explode = (0.02, 0.02) # creating explode data

# Creating autocpt arguments
def func(pct, allvalues):
    absolute = int(pct / 100.*np.sum(allvalues))
    return "{:.1f}%\n({:d})".format(pct, absolute)

# Creating plot
plt.figure(figsize=(18, 8))
plt.subplot(1, 2, 1)
plt.pie(data, labels=labels, explode=explode, colors=colors, 
                                  autopct=lambda pct: func(pct, data), startangle=90, textprops=dict(color ="#3F1D38"))

# Adding legend
plt.legend(labels, title ="Years", loc ='best',)

plt.title("Number of offences distrubuted by YEAR")

# Bat chart
color=['#B5F1CC']
plt.subplot(1, 2, 2)
order = df['INCIDENT_MONTH'].value_counts().sort_index().index
ax = sns.countplot(x='INCIDENT_MONTH', data=df, order=order, palette=color)
for label in ax.containers[0]:
    ax.annotate(format(int(label.get_height())), 
                (label.get_x() + label.get_width() / 2., label.get_height()), 
                ha='center', va='center', xytext=(0, 9), textcoords='offset points')
    
plt.title('Number of Offences Distributed by Months')
plt.xlabel('Months')

plt.tight_layout() 
plt.show()

In [ ]:
plt.figure(figsize=(20, 6))

# Bar chart for dates
order = df['INCIDENT_DATE'].value_counts().sort_index().index
ax = sns.countplot(x='INCIDENT_DATE',data=df, order=order, color='#7C9D96')

for label in ax.containers[0]:
    ax.annotate(format(int(label.get_height())), 
                (label.get_x() + label.get_width() / 2., label.get_height()), 
                ha='center', va='center', xytext=(0, 9), textcoords='offset points')

plt.title('Number of Offences Distributed by Dates')
plt.xlabel('Dates')
sns.set(font_scale=1.25)

plt.show()

In [ ]:
# Bar chart for weekdays
plt.figure(figsize=(20, 6))

# Bar chart for dates
order = df['INCIDENT_WEEKDAY'].value_counts().sort_index().index
ax = sns.countplot(x='INCIDENT_WEEKDAY',data=df, order=order, color='#EEE0C9')

for label in ax.containers[0]:
    ax.annotate(format(int(label.get_height())), 
                (label.get_x() + label.get_width() / 2., label.get_height()), 
                ha='center', va='center', xytext=(0, 9), textcoords='offset points')

plt.title('Number of Offences Distributed by Weekday')
plt.xlabel('Weekday')

plt.show()

In [ ]:
plt.figure(figsize=(20, 6))

# Plot the count of offenses by time
order = df['INCIDENT_TIME'].value_counts().sort_values().index
ax = sns.countplot(x='INCIDENT_TIME', data=df, color='#EEE0C9', order=order)

for label in ax.containers[0]:
    ax.annotate(format(int(label.get_height())), 
                (label.get_x() + label.get_width() / 2., label.get_height()), 
                ha='center', va='center', xytext=(0, 9), textcoords='offset points')

plt.title('Number of Offenses Distributed by Time')
plt.xlabel('Time')
plt.ylabel('Count')

plt.xticks(rotation=45)  # Rotate x-axis labels for better readability

plt.show()

In [ ]:
df.head()

In [ ]:
# Creating data
data = [len(df[df['CRIME_TYPE'] == feature]) for feature in df['CRIME_TYPE'].value_counts().index[0:5]]
labels = df['CRIME_TYPE'].value_counts().index[0:5].tolist()

colors = ['#A1CCD1', '#FFC6AC', '#FFF6DC', '#C4C1A4', '#9E9FA5'] # creating color parameters
explode = (0.02, 0.02, 0.02, 0.02, 0.02) # creating explode data

# Creating autocpt arguments
def func(pct, allvalues):
    absolute = int(pct / 100.*np.sum(allvalues))
    return "{:.1f}%\n({:d})".format(pct, absolute)

# Creating plot
plt.figure(figsize=(15, 6))
plt.subplot(1, 2, 1)
plt.pie(data, explode=explode, colors=colors, labels=labels,
        autopct=lambda pct: func(pct, data), startangle=90, textprops={'color': "#3F1D38", 'fontsize': 9})
plt.title("Number of offences distrubuted by crimes")

# Adding legend
# plt.legend(labels, title ="Type of crimes",
#           loc ="center left",
#           bbox_to_anchor =(1, 0.1, 0.5, 0.5))


data1 = [len(df[df['INCIDENT_REASON'] == feature]) for feature in df['INCIDENT_REASON'].value_counts().index[0:5]]
labels1 = df['INCIDENT_REASON'].value_counts().index[0:5].tolist()

colors = ['#A1CCD1', '#FFC6AC', '#FFF6DC', '#C4C1A4', '#9E9FA5'] # creating color parameters
explode = (0.02, 0.02, 0.02, 0.02, 0.02) # creating explode data

# Creating autocpt arguments
def func(pct, allvalues):
    absolute = int(pct / 100.*np.sum(allvalues))
    return "{:.1f}%\n({:d})".format(pct, absolute)

# Creating plot
plt.subplot(1, 2, 2)
plt.pie(data1, explode=explode, colors=colors, labels=labels1,
        autopct=lambda pct: func(pct, data1), startangle=90, textprops={'color': "#3F1D38", 'fontsize': 9})

# Adding legend
# plt.legend(labels1, title ="Type of crimes",
#           loc ="center left",
#           bbox_to_anchor =(1, 0.1, 0.5, 0.5))

plt.show()

In [ ]:
# Bar chart: Location district
plt.figure(figsize=(15, 7))
order = df['LOCATION_DISTRICT'].value_counts().index[:20]
ax = sns.countplot(x=df['LOCATION_DISTRICT'], order=order, color='#D0BFFF')

for label in ax.containers:
    ax.bar_label(label)

plt.xlabel('Location District')

plt.show()

In [ ]:
df['INCIDENT_YEAR'] = df['INCIDENT_YEAR'].astype(str)

# Step 3: Feature Engineering

In [ ]:
# Label encoding
categorical_features = df.select_dtypes(include=['object', 'category'])

for feature in categorical_features:
    le = LabelEncoder()
    df[feature] = le.fit_transform(df[feature])
    
df.head()

In [ ]:
df.dtypes

# Step 4: Dimensionality Reduction

### Principal Component Analysis

In [ ]:
scaled_data = StandardScaler().fit_transform(df)
pca = PCA()
principal_components = pca.fit_transform(scaled_data)
pca.explained_variance_

In [ ]:
scaled_df = pd.DataFrame(scaled_data, columns=df.columns)

In [ ]:
var_exp = pca.explained_variance_ratio_ * 100
print(np.round_(var_exp, decimals = 5))

In [ ]:
cum_var_exp = np.cumsum(var_exp)
print(cum_var_exp)

In [ ]:
# extract the Eigenvectors
eig_vecs = pca.components_
eig_vecs

In [ ]:
var = pca.explained_variance_ 
labels = list(range(0, len(var)))  # Creating labels for each principal component

plt.figure(figsize=(18, 7))

# Creating the bar plot
ax = plt.bar(labels, var, color="#DFA67B")

# Adding labels on top of each bar
for index, value in enumerate(var):
    plt.text(index, value, f"{value:.2f}", ha='center', va='bottom')

plt.xlabel('Principal Component')
plt.ylabel('Proportion of Variance Explained')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(20,10))

x = np.arange(1, 11, step=1)
y = np.cumsum(pca.explained_variance_ratio_)

plt.plot(x, y, marker='o', linestyle='--', color='b')
plt.xlabel('Number of Components')
plt.xticks(np.arange(0, 11, step=1))
plt.ylabel('Cumulative variance (%)')

plt.title('The number of components needed to explain variance')
plt.axhline(y=0.90, color='r', linestyle='-')
plt.text(0.5, 0.85, '90% cut-off threshold', color='red', fontsize=12)

plt.show()

In [ ]:
# use PCA to project the data 
n_components = 6 # gives 96%
pca = PCA(n_components=n_components)  # Retain 96% of variance
reduced_data = pca.fit_transform(scaled_data)
reduced_data

### PCA as Feature Selection

In [ ]:
top_components = pca.components_[:n_components]

# Calculate the absolute values of loadings and sort features by importance
feature_importance = np.abs(top_components).sum(axis=0)
sorted_feature_indices = np.argsort(feature_importance)[::-1]

# Select the top features
selected_feature_indices = sorted_feature_indices[:n_components]
selected_features = scaled_df.columns[selected_feature_indices]
selected_features

### Feature Variance

In [ ]:
selector_ = VarianceThreshold(threshold=4)
selector_.fit_transform(df)
cols_ind = selector_.get_support(indices=True)
cols_names = df.columns
selected_cols_names = [cols_names[i] for i in cols_ind]
selected_cols_names

# Step 5: Cluster Analysis

### Choosing the Number of Clusters k

In [ ]:
# create a k-Means model an Elbow-Visualizer
model = KMeans()
visualizer = KElbowVisualizer(model, k=(1, 10), timings=False)
 
# fit the visualizer and show the plot
visualizer.fit(reduced_data)
visualizer.show()

### K-Means clustering

In [ ]:
# clustering
kmeans = KMeans(n_clusters=4, random_state=0).fit(reduced_data)
 
# extract centroids of clusters into a dataframe
centers = kmeans.cluster_centers_

# extract cluster labels
labels = kmeans.labels_ 
scaled_df["Labels"] = labels

In [ ]:
# creating dataframe for reduced data 
reduced_df = pd.DataFrame(reduced_data, columns=selected_features)
reduced_df['CLUSTER'] = labels
reduced_df['CLUSTER'] = reduced_df['CLUSTER'].astype("category")
reduced_df.head()

### Silhouette Score & Davies-Bouldin Index & Variance Ratio Criterion & Within-Cluster Sum of Squares (WCSS)

In [ ]:
silhouette_avg = silhouette_score(reduced_data, kmeans.labels_)
print('Silhouette Score', silhouette_avg)

db_index = davies_bouldin_score(reduced_data, kmeans.labels_)
print('Davies-Bouldin Index:', db_index)

ch_score = calinski_harabasz_score(reduced_data, kmeans.labels_)
print('Calinski-Harabasz Index (Variance Ratio Criterion):', ch_score)

wcss = kmeans.inertia_
print('Within-Cluster Sum of Squares (WCSS):', wcss)

In [ ]:
# Visualization of kmeans with TSNE

tsne = TSNE(n_components=2, random_state=0)
projections = tsne.fit_transform(scaled_df)

plt.figure(figsize=(8, 6))
for i in range(4):
    plt.scatter(projections[labels == i][:, 0], projections[labels == i][:, 1], label=f'Cluster {i + 1}')
    
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.title('K-means Clustering with t-SNE')
plt.legend(loc ='best')
plt.show()

In [ ]:
# Visualization of kmeans with UMAP

umap = umap.UMAP(n_components=2, random_state=0, init='random')
umap_projections = umap.fit_transform(scaled_df)

plt.figure(figsize=(8, 6))
for i in range(4):
    plt.scatter(umap_projections[labels == i][:, 0], umap_projections[labels == i][:, 1], label=f'Cluster {i + 1}')
    
plt.xlabel('UMAP Component 1')
plt.ylabel('UMAP Component 2')
plt.title('K-means Clustering with UMAP')
plt.legend(loc ='best')
plt.show()

In [ ]:
# Visualization of kmeans with PCA

plt.figure(figsize=(10, 12))
ax = plt.axes(projection ="3d")

for i in range(4):
    ax.scatter3D(reduced_data[labels == i][:, 0], reduced_data[labels == i][:, 1], 
                reduced_data[labels == i][:, 2], label=f'Cluster {i + 1}', s=80)
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')
plt.title('K-means Clustering with PCA')
plt.legend(loc ='best')
plt.show()

In [ ]:
cor_mat = df.corr(method='spearman')

plt.figure(figsize=(15, 7))
sns.set(font_scale=0.6)
mask = np.triu(np.ones_like(cor_mat, dtype=np.bool))
ax = sns.heatmap(cor_mat, annot=True, fmt=".2g", vmin=-1, vmax=1,
                annot_kws={'size': 'medium'}, linewidths=0.8, mask=mask)
plt.show()

In [ ]:
sns.pairplot(hue="CLUSTER", data=reduced_df)
plt.show()